In [ ]:
import pandas as pd
import numpy as np
import scimap as sm
import anndata as ad
import warnings
warnings.filterwarnings("ignore")

# Prior knowledge driven annotation

If you did Clustering before, use your anndata object from clustering!

In [ ]:
adata = ad.read_h5ad('/Users/lukashat/Documents/PhD_Schapiro/Teaching/SysBio/2025/src/cells_annotated.h5ad')

## read in decision matrix and inspect

In [ ]:
# load the phenotyping workflow
phenotype = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Teaching/SysBio/2025/src/scimapExampleData/phenotype_workflow.csv')
# view the table:
phenotype.style.format(na_rep='')

Very 'broad' phenotypes only here. Try to undertand how this matrix works and how it tries to recapitulate the hierarchical system we talked about

In [ ]:
#%gui qt

In [ ]:
image_path = '/Users/lukashat/Documents/PhD_Schapiro/Teaching/SysBio/2025/src/scimapExampleData/registration/exemplar-001.ome.tif'

## Threshold the image
This is very subjective and the data is not really 'clean' and suffers a lot from Autofluorescence. Try to find good gates by looking at the marker on the image and the histograms, add biological knowledge (long bright lines are rather autofluorescent elastic fibrers than real staining for example)

In [ ]:
sm.pl.napariGater (image_path, adata)

In [ ]:
adata

In [ ]:
adata = sm.pp.rescale (adata, gate=adata.uns['gates'])

In [ ]:
adata = sm.tl.phenotype_cells (adata, phenotype=phenotype, label="phenotype") 

In [ ]:
adata.obs['phenotype'].value_counts()

## Now rename the cells so they fit to the cluster annotations (hint: Dendritic cells need to 'vanish' into another category, otherwise check markers from clustering). This is needed for step 3

In [ ]:
adata.obs.dtypes

In [ ]:
adata.obs['phenotype'].value_counts()

In [ ]:
adata.obs['leiden_phenotype'].value_counts()

In [ ]:
adata.obs['phenotype'] = adata.obs['phenotype'].astype(str)
adata.obs['phenotype_to_compare'] = adata.obs['phenotype']
adata.obs['phenotype_to_compare'] = adata.obs['phenotype_to_compare'].replace({'ECAD+': 'Tumor', 'SMA+': 'Vessels', 'Other myeloid cells': 'Myeloid',
                                                                               'Dendritic cells': 'Myeloid', 'NK cells': 'Immune'})

In [ ]:
adata.write_h5ad('/Users/lukashat/Documents/PhD_Schapiro/Teaching/SysBio/2025/src/cells_annotated_both.h5ad')